# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity(%),Cloudiness(%),Wind Speed(MPH),Country,Data
0,0,Barrow,71.2906,-156.7887,1.40,78,90,23.02,US,1610465022
1,1,Phalombe,-15.8064,35.6507,76.24,88,100,3.56,MW,1610465022
2,2,Thinadhoo,0.5333,72.9333,82.02,78,100,18.01,MV,1610465022
3,3,Kapaa,22.0752,-159.3190,66.20,94,20,8.05,US,1610465022
4,4,Albany,42.6001,-73.9662,34.00,66,100,6.55,US,1610465023
...,...,...,...,...,...,...,...,...,...,...
590,590,Irará,-12.0500,-38.7667,84.33,60,66,8.86,BR,1610465144
591,591,Batamshinskiy,50.5594,58.2747,-9.40,70,0,4.47,KZ,1610465144
592,592,Natchitoches,31.7668,-93.1335,35.60,93,1,2.13,US,1610465145
593,593,Chenghai,23.4632,116.7680,46.40,61,0,4.47,CN,1610465145


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

# use humidity as the weights for the heat map

In [6]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Locations and Weights
locations=cities_df[['Lat', 'Lng']]
humidity=cities_df['Humidity(%)']

#Heatmap
fig=gmaps.figure()
heatmap=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
#Create dataframe for cities with perfect conditions
df_new= cities_df[(cities_df['Max Temp (F)'] < 80) 
                   & (cities_df['Max Temp (F)'] > 70) 
                   & (cities_df['Wind Speed(MPH)'] < 10) 
                   & (cities_df['Cloudiness(%)'] == 0)]
df_new                

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity(%),Cloudiness(%),Wind Speed(MPH),Country,Data
71,71,Viedma,-40.8135,-62.9967,75.00,18,0,1.01,AR,1610465035
155,155,Dwārka,22.2394,68.9678,70.84,48,0,7.63,IN,1610465054
186,186,Sechura,-5.5569,-80.8222,78.53,58,0,9.60,PE,1610465059
250,250,Necochea,-38.5473,-58.7368,71.04,27,0,9.06,AR,1610464880
411,411,Pisco,-13.7000,-76.2167,73.40,73,0,6.91,PE,1610465104
499,499,Alice Springs,-23.7000,133.8833,77.00,38,0,8.46,AU,1610465097
529,529,Córdoba,-31.4135,-64.1811,78.01,40,0,4.61,AR,1610464887
570,570,Sompeta,18.9333,84.6000,72.99,77,0,0.58,IN,1610465140
580,580,Dohad,22.8333,74.2667,70.43,33,0,7.54,IN,1610465142
582,582,Acajutla,13.5928,-89.8275,78.80,50,0,5.75,SV,1610465143


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
#  Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df=df_new[["City","Country","Lat", "Lng"]].copy()
hotel_df['Hotel Name']= ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
71,Viedma,AR,-40.8135,-62.9967,
155,Dwārka,IN,22.2394,68.9678,
186,Sechura,PE,-5.5569,-80.8222,
250,Necochea,AR,-38.5473,-58.7368,
411,Pisco,PE,-13.7000,-76.2167,
499,Alice Springs,AU,-23.7000,133.8833,
529,Córdoba,AR,-31.4135,-64.1811,
570,Sompeta,IN,18.9333,84.6000,
580,Dohad,IN,22.8333,74.2667,
582,Acajutla,SV,13.5928,-89.8275,


In [17]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
# get lat, lng from df
    latitude=row['Lat']
    longitude=row['Lng']
    coords=f'{latitude}, {longitude}'
    params["location"] = coords
# Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
# make request and print url
    name_address = requests.get(base_url, params=params)
    results = response['results']


   # Grab the first hotel from the results and store the name
    try:
        hotel_name = hotel_df.loc[index, 'Hotel Name'] =results[0]['name']
        
    except:
        continue
        


,City,Country,Lat,Lng,Hotel Name
71,Viedma,AR,-40.8135,-62.9967,Hotel Cristal
155,Dwārka,IN,22.2394,68.9678,Hotel Narayan Avezika Comfort
186,Sechura,PE,-5.5569,-80.8222,Hospedaje Costa Bella
250,Necochea,AR,-38.5473,-58.7368,dyd -mar Hotel
411,Pisco,PE,-13.7000,-76.2167,Hotel San Isidro Oficial


In [18]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
71,Viedma,AR,-40.8135,-62.9967,Hotel Cristal
155,Dwārka,IN,22.2394,68.9678,Hotel Narayan Avezika Comfort
186,Sechura,PE,-5.5569,-80.8222,Hospedaje Costa Bella
250,Necochea,AR,-38.5473,-58.7368,dyd -mar Hotel
411,Pisco,PE,-13.7000,-76.2167,Hotel San Isidro Oficial
499,Alice Springs,AU,-23.7000,133.8833,Desert Palms Alice Springs
529,Córdoba,AR,-31.4135,-64.1811,Sheraton Cordoba Hotel
570,Sompeta,IN,18.9333,84.6000,బెంకిలి భైరొడమ్మ క్రికెట్ గ్రౌండ్
580,Dohad,IN,22.8333,74.2667,Hotel Laxmi Inn
582,Acajutla,SV,13.5928,-89.8275,asi es


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))